In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import numpy as np
import plotly.express as px

In [2]:
path = "C:\\Users\\Tiziano Pacifico\\Desktop\\TESI\\RedditDS\\chatGPT\\final_df\\"
filename = "final_df.json"

In [3]:
df = pd.read_json(path+filename, lines=True)

In [4]:
df.tail(1)

,author,created_utc,num_comments,utc_datetime_str,text,id,BERT_out,GPT_label,subjectivity,polarity
73942,hi_there_bitch,1669925968,5466,2022-12-01 20:19:28,r/ChatGPT LoungeA place for members of r/ChatG...,z9yhzj,"[[like, 0.0201793395], [use, 0.0190240443], [a...",AI and ChatGPT in Everyday Life,0.467052,0.055139


In [38]:
#Polarità media per ogni topic
df_grouped = df.groupby('GPT_label')['polarity'].mean().reset_index().rename(columns={'polarity': 'average_polarity'})

In [80]:
#Conteggio del numero di documenti per topic
df_counter = df.groupby('GPT_label').size().reset_index(name = 'document counts')

In [82]:
df_grouped_orderer = df_grouped.sort_values(by = 'average_polarity', ascending = False).reset_index().drop(columns=['index'])

In [83]:
#Aggiungo la colonna con il conteggio al dataset che rappresenta la media della polarità per topic
df_grouped_orderer = df_grouped_orderer.merge(df_counter, how='left', on='GPT_label')

In [97]:
df_grouped_orderer.head(1)

,GPT_label,average_polarity,document counts
0,Managing Discord Server Interactions with GPT-...,0.166176,576


In [91]:
#Bar plot della polarità media per topic

#N = 22    # Number of boxes
#c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]
fig = go.Figure()
fig.add_trace(go.Bar(
    y=df_grouped_orderer['GPT_label'],
    x=df_grouped_orderer['average_polarity'],
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.update_layout(
        title=dict(
        text='<b>Polarity media per topic',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=24,
            color='black',
            family="Arial, sans-serif",
        )
    ),
    height=600)
fig.show()

In [49]:
#Elenco dei topic secondo l'ordine di polarità media
topics = df_grouped_orderer['GPT_label']

In [50]:
#Per ogni topic estraggo una lista con tutti i valori di polarità, mi servirà per rappresentare la dispersione dei valori
topics_polarity_lists = []
for topic in topics:
    topics_polarity_list = list(df[df['GPT_label'] == topic]['polarity'])
    topics_polarity_lists.append(topics_polarity_list)

In [51]:
N = len(topics_polarity_lists)     # Number of boxes
c = ['hsl('+str(h)+',50%'+',50%)' for h in np.linspace(0, 360, N)]

In [52]:
range_df = len(topics_polarity_lists)

In [93]:
fig = go.Figure(data=[go.Box(
    y=topics_polarity_lists[i],
    marker_color=c[i],
    name=topics[i]
    ) for i in range(range_df)])
# format the layout
fig.update_layout(
        title=dict(
        text='<b>Dispersione della polarità per topic',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=24,
            color='black',
            family="Arial, sans-serif",
        )
    ),
    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    yaxis=dict(zeroline=False, gridcolor='white'),
    paper_bgcolor='rgb(233,233,233)',
    plot_bgcolor='rgb(233,233,233)',
)
fig.update_layout(height=700)
fig.update_yaxes(title_text="Sentiment polarity range")
fig.show()

In [87]:
#Definisco una sentiment negativo/positivo
polarity_values = df[['polarity','GPT_label']].copy()

In [88]:
def polarise(value):
    if value >= 0:
        return 1
    else:
        return 0

In [89]:
#Per ogni documento riclassifico i valori di polarità come negativa o positiva
polarity_values['binary_polarity'] = polarity_values['polarity'].apply(polarise)

In [23]:
#Conto le occorrenze di polarità positiva e negativa per ogni topic
grouped = polarity_values.groupby('GPT_label')['binary_polarity'].value_counts().unstack().fillna(0)

In [63]:
grouped['total'] = grouped[0]+grouped[1]

In [64]:
grouped = grouped.sort_values(by='total', ascending=False)

In [65]:
grouped = grouped.drop('total',axis=1)

In [94]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=grouped.index,
    x=grouped[0],
    name='Negative sentiment',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=grouped.index,
    x=grouped[1],
    name='Positive sentiment',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))
fig.update_layout(
        title=dict(
        text='<b>Numero di documenti per topic con polarità negativa/positiva',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=24,
            color='black',
            family="Arial, sans-serif",
        )
    ),
    legend=dict(
        title="Legenda",  # Titolo della legenda
        yanchor="top",  # Posizione verticale della legenda
        y=1.0,  # Posizione verticale della legenda
        xanchor="right",  # Posizione orizzontale della legenda
        x=-1  # Posizione orizzontale della legenda
    )
)
fig.update_layout(barmode='group', height=800, width=1000)
fig.update_xaxes(title_text="Post count")
fig.show()

In [34]:
#Calcolo la magnitudo della polarità sommando tutti i valori della stessa polarità per ogni topic
result = polarity_values.groupby(['GPT_label', 'binary_polarity'])['polarity'].sum().unstack()
#result.rename(columns={0: 'SUM_0', 1: 'SUM_1'}, inplace=True)

In [126]:
result = result.sort_values(by=1, ascending=False)

In [106]:
result.head(1)

binary_polarity,0,1
GPT_label,,
AI Bots Managing Online Interactions in Chat Platforms,-17.246761,390.103311


In [100]:
#Valori medi della polarità negativa e positiva per ogni topic
result_2 = polarity_values.groupby(['GPT_label', 'binary_polarity'])['polarity'].mean().unstack()

In [127]:
fig = go.Figure()
fig.add_trace(go.Bar(
    y=result.index,
    x=result[0],
    name='Negative sentiment',
    orientation='h',
    marker=dict(
        color='rgba(246, 78, 139, 0.6)',
        line=dict(color='rgba(246, 78, 139, 1.0)', width=3)
    )
))
fig.add_trace(go.Bar(
    y=result.index,
    x=result[1],
    name='Positive sentiment',
    orientation='h',
    marker=dict(
        color='rgba(58, 71, 80, 0.6)',
        line=dict(color='rgba(58, 71, 80, 1.0)', width=3)
    )
))
fig.update_layout(
        title=dict(
        text='<b>Magnitudo della polarità negativa e positiva per topic',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=24,
            color='black',
            family="Arial, sans-serif",
        )
    ),
    legend=dict(
        title="Legenda",  # Titolo della legenda
        yanchor="top",  # Posizione verticale della legenda
        y=1.0,  # Posizione verticale della legenda
        xanchor="right",  # Posizione orizzontale della legenda
        x=-1  # Posizione orizzontale della legenda
    )
)
fig.update_layout(barmode='group', height=800, width=1000)
fig.update_xaxes(title_text="Total sentiment")
fig.show()

In [112]:
df_grouped_2 = df_grouped_orderer.merge(result_2.reset_index(), how='left',on='GPT_label')

In [117]:
df_grouped_2 = df_grouped_2.fillna(0)

In [118]:
df_grouped_2.head(1)

,GPT_label,average_polarity,document counts,0,1
0,Managing Discord Server Interactions with GPT-...,0.166176,576,0.0,0.166176


In [125]:
fig = go.Figure()

# Aggiungi il primo scatterplot
fig.add_trace(go.Scatter(x=df_grouped_2['document counts'], y=df_grouped_2['average_polarity'], mode='markers', name='Total'))

# Aggiungi il secondo scatterplot
fig.add_trace(go.Scatter(x=df_grouped_2['document counts'], y=df_grouped_2[0], mode='markers', name='Negative'))

# Aggiungi il terzo scatterplot
fig.add_trace(go.Scatter(x=df_grouped_2['document counts'], y=df_grouped_2[1], mode='markers',name='Positive'))

# Aggiungi titolo e etichette agli assi
fig.update_layout( title=dict(
        text='<b>Scatter della polarità totale, negativa e positiva rispetto alla dimensione dei topic',  # Title text
        x=0.5,  # x=0.5 centers the title
        xanchor='center',  # 'center' ensures the title is centered
        font=dict(
            size=24,
            color='black',
            family="Arial, sans-serif",
        )
    ),
                  xaxis_title='N doc per topic',
                  yaxis_title='Polarity')

# Mostriamo il grafico
fig.show()